In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 https://developer.downlo

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()


In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Home_v1_00.tsv.gz"), sep="\t", header=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   33670092|R1UUISQ1GKOJTI|B00EE62UAE|     583436067|Trademark Home Po...|            Home|          1|            0|          0|   N|                Y|                 Run|Don't buy it clap...| 2015-08-31|
|         US|   13726692|R1HOJ9WE8VCVOD|B001APXO5C|     465035091|O2-Cool 10-Inch P...|            Home|          5|    

In [4]:
df2 = df.select(["star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
df2.show(10)

df3 = df2.filter(df2["total_votes"] >= 20)
df4 = df3.filter(df3["helpful_votes"]/df3["total_votes"] >= 0.5)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          1|            0|          0|   N|                Y|
|          5|            9|          9|   N|                Y|
|          5|            0|          0|   N|                Y|
|          2|            0|          0|   N|                Y|
|          4|            0|          0|   N|                N|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          1|            0|          0|   N|                Y|
|          2|            0|          0|   N|                Y|
+-----------+-------------+-----------+----+-----------------+
only showing top 10 rows



In [5]:
# Filter the data and create a new DataFrame where total_votes >= 20
vine_df = df.filter("total_votes>=20").select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R27XX9MG7OWSSM|          5|           49|         51|   N|                Y|
|R1I4IFDOW6IW9O|          3|           44|         47|   N|                Y|
|R2LZAV3VMLGPGS|          3|           32|         33|   N|                Y|
|R3FITR2BQI94RA|          5|           33|         38|   N|                Y|
| RWYBT1R3KPD2M|          4|          142|        163|   N|                Y|
| RE4023DS92U8H|          5|           26|         27|   N|                Y|
|R25TLB0EXZ7DJZ|          5|           45|         46|   N|                Y|
|R3BEQIBG1OQ7RT|          2|            3|         22|   N|                Y|
|R1SQKXYWU7VMW1|          2|           23|         24|   N|                Y|
| RUL6FPHYI15TA|          2|           62|         70|   N|     

In [6]:
Tvine_df = vine_df.filter("helpful_votes/total_votes>=.5").select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
Tvine_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R27XX9MG7OWSSM|          5|           49|         51|   N|                Y|
|R1I4IFDOW6IW9O|          3|           44|         47|   N|                Y|
|R2LZAV3VMLGPGS|          3|           32|         33|   N|                Y|
|R3FITR2BQI94RA|          5|           33|         38|   N|                Y|
| RWYBT1R3KPD2M|          4|          142|        163|   N|                Y|
| RE4023DS92U8H|          5|           26|         27|   N|                Y|
|R25TLB0EXZ7DJZ|          5|           45|         46|   N|                Y|
|R1SQKXYWU7VMW1|          2|           23|         24|   N|                Y|
| RUL6FPHYI15TA|          2|           62|         70|   N|                Y|
|R1K2O880XPCW4V|          2|           26|         27|   N|     

In [7]:
vineYES_df = Tvine_df.filter("vine== 'Y'").select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vineYES_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2W1OMYJERBXJ1|          5|           20|         23|   Y|                N|
| R31VJCBBKU13Y|          5|           24|         27|   Y|                N|
|R350JNDV0INC11|          4|          388|        398|   Y|                Y|
|R2NJA2I5C9Y0FR|          3|           63|         73|   Y|                N|
|R296RSG25RD1IX|          3|           46|         51|   Y|                N|
|R3IJKA4RI56TFT|          4|           25|         30|   Y|                N|
|R1EU2LNHAWWPFI|          5|           19|         24|   Y|                N|
|R164L6F7QRHL8A|          5|           94|        109|   Y|                N|
|R3VONEXS4LRVWK|          3|          378|        388|   Y|                N|
| RYO3NDB745QIW|          5|           33|         37|   Y|     

In [8]:
vineNO_df = Tvine_df.filter("vine== 'N'").select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vineNO_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R27XX9MG7OWSSM|          5|           49|         51|   N|                Y|
|R1I4IFDOW6IW9O|          3|           44|         47|   N|                Y|
|R2LZAV3VMLGPGS|          3|           32|         33|   N|                Y|
|R3FITR2BQI94RA|          5|           33|         38|   N|                Y|
| RWYBT1R3KPD2M|          4|          142|        163|   N|                Y|
| RE4023DS92U8H|          5|           26|         27|   N|                Y|
|R25TLB0EXZ7DJZ|          5|           45|         46|   N|                Y|
|R1SQKXYWU7VMW1|          2|           23|         24|   N|                Y|
| RUL6FPHYI15TA|          2|           62|         70|   N|                Y|
|R1K2O880XPCW4V|          2|           26|         27|   N|     

In [9]:
ratingsPaid = vineYES_df.filter("star_rating==5").count()
print(f"The total number of Paid 5-star ratings : {ratingsPaid}")

The total number of Paid 5-star ratings : 647


In [10]:
countYes = vineYES_df.count()
print(f"The total number of Paid reviews : {countYes}")

The total number of Paid reviews : 1448


In [11]:
ratingsUnPaid = vineNO_df.filter("star_rating==5").count()
print(f"The total number of Unpaid 5-star ratings : {ratingsUnPaid}")

The total number of Unpaid 5-star ratings : 44104


In [12]:
total5star = ratingsPaid + ratingsUnPaid
print(f"The total number of paid and Unpaid 5-star ratings : {total5star}")

The total number of paid and Unpaid 5-star ratings : 44751


In [13]:
countNO = vineNO_df.count()
print(f"The total number of Unpaid reviews : {countNO}")

The total number of Unpaid reviews : 90768


In [14]:
totalReviewers = countYes + countNO
print(f"The total number of Paid and and Unpaid reviewers : {totalReviewers}")

The total number of Paid and and Unpaid reviewers : 92216


In [15]:
percentagePaid = (ratingsPaid/totalReviewers)*100
print(f"The percentage of paid ratings : {percentagePaid}%")

The percentage of paid ratings : 0.7016136028454932%


In [16]:
percentageUnPaid = (ratingsUnPaid/totalReviewers)*100
print(f"The percentage of unpaid ratings : {percentageUnPaid}%")

The percentage of unpaid ratings : 47.82684132905353%


In [17]:
percentPaid = (ratingsPaid/countYes)*100
print(f"The percentage of paid ratings : {percentPaid}%")

The percentage of paid ratings : 44.68232044198895%


In [18]:
percentUnPaid = (ratingsUnPaid/countNO)*100
print(f"The percentage of unpaid ratings : {percentUnPaid}%")

The percentage of unpaid ratings : 48.589811387273045%
